<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/main/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
from bs4 import BeautifulSoup as b4
import asyncio
import os
import multiprocessing
import time
import requests
!pip install selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [59]:
TRIAL = 3

### Parse chapter

In [88]:
def parseFile(html):
  #with open(html, "r") as file:
  soup = b4(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  content = ""
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Download main webpage

In [113]:
def getChapter(url, browser, trial):
  try:
      browser.get(url)
      print(f"attempting to download url: {url}")
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
      parseFile(html)
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
      else:
        print(f"Download failed trial {trial}")
        getChapter(url, browser, trial + 1)

### Get webpage

In [130]:
def getNovelPage(url, browser, trial):
  try:
      print(f"getNovelPage trying url: {url}...")
      browser.get(url)
      html = browser.page_source
      soup = b4(html, features="html.parser")
      strainer = soup.find(class_="chapter-list").find_all('a')
      for link in strainer:
        getChapter("https://www.lightnovelpub.com"+ link['href'], browser, 0)
      return html
  except TimeoutException:
      if trial >= TRIAL:
        with open(url, "w") as file:
          file.write("I DONE GOOFED!!!")
        return -1
        file.close()
        print("I give up...")
      else:
        getNovelPage(url, browser, trial + 1)
  finally:
      browser.quit()

### Get links and go to next chapter page, if avaliable (testing from file)

In [122]:
def getChapterPages(url, page_num):
  # Start a new selenium process
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)

  # Get the novel main page
  url_source = getNovelPage(url, browser, 0)

  if url_source == -1:
    print("Unable to find page source!")
    return

  soup = b4(url_source, features="html.parser")
  links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
  next_url = ""
  for link in links:
    next_url =  link.find('a')['href']
  if next_url:
    # Make new url
    new_url = "https://www.lightnovelpub.com" + next_url
    getChapterPages(new_url, page_num+1)
  else:
    browser.quit()

### Input url

In [129]:
url = input("Enter url: ")
directory_name = input("Enter directory name: ")

# Create the directory
os.mkdir(directory_name)

# Navigate into the directory
os.chdir(directory_name)

getChapterPages(url, 1)

getNovelPage trying url: https://www.lightnovelpub.com/novel/chrysalis-16091353/chapters...
attempting to download url: https://www.lightnovelpub.com/novel/chrysalis-552/chapter-1-30041322
[<p>I had no time to think, pause or consider before I came to be enveloped in a soft, white light. A blinding flash impacting not just my eyes but seemingly my whole body.</p>, <p>Sensation fell gradually away as I tried to gather my thoughts. What had happened? Where was I? I think I made myself dinner, my parents hadn’t been home, again. I was eating in my room, at my desk maybe? Then there had been ... white?</p>, <p>I can’t tell how long I’ve been here, floating in this gentle light. Has it been ten days? Or only ten minutes? The light surrounds me, enveloping me like a warm blanket.</p>, <p>Sound begins to swell like a wave in the distance. It rushes toward me and just when washes over me I hear a voice.</p>, <p>[Welcome Anthony]</p>, <p>Well thank you! These are the kinds of good manners that 

In [2]:
import os

# Ask the user for the directory name
directory_name = input("Enter the directory name: ")

# Create the directory
os.mkdir(directory_name)

# Navigate into the directory
os.chdir(directory_name)

# Print the current working directory to verify
print("Current working directory:", os.getcwd())

# Create and write to a file
with open("asdf.txt", "w") as file:
    file.write("best")


Enter the directory name: afd
Current working directory: /content/afd
